In [ ]:
import os
import re
import glob
import json
import pickle

from tqdm import tqdm
import datetime as dt
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
PROFILE_FILTERED_PATH = "../data/supports/profile_filtered.pkl"
USER_TIMELINE_PATH = "../data/user_timeline"
CHECK_LATEST_TWEET_PATH = "../data/supports/check_latest_tweet.pkl"

In [ ]:
d_profile = pickle.load(open(PROFILE_FILTERED_PATH, 'rb'))

In [ ]:
tqdm.pandas()

In [ ]:
def parse_created(date):
    date = re.sub(" \+[0-9]{0,4}", "", date)
    date = re.sub("^[a-zA-Z]{3} ", "", date)
    date = re.sub("[0-9]{2}:[0-9]{2}:[0-9]{2}", "", date)
    
    tgl = re.search("[0-9]{2}", date).group()
    bln = re.search("[A-Za-z]{3}", date).group()
    bln = month_dict[bln]
    thn = re.search("[0-9]{4}", date).group()
    date = tgl + "-" + bln + "-" + thn
    
    date = datetime.strptime(date, "%d-%m-%Y")
    
    return date

In [ ]:
month_dict = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06"
             , "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"}

In [ ]:
def read_tweet(screen_name):
    file = os.path.join(USER_TIMELINE_PATH, screen_name + '.json')
    if os.path.exists(file):
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        date = parse_created(data["created_at"])
        return date
    else:
        return None

In [ ]:
d_profile["latest_tweet"] = d_profile.screen_name.progress_apply(read_tweet)

In [ ]:
d_profile["duration_latest_tweet"] = d_profile.latest_tweet.apply(
    lambda x: (datetime.today() - x).days)

In [ ]:
d_profile = d_profile[~d_profile.duration_latest_tweet.isna()]

In [ ]:
d_profile.shape

In [ ]:
pickle.dump(d_profile, open(CHECK_LATEST_TWEET_PATH, 'wb'))